In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

from rdkit import Chem
from sklearn import metrics

In [ ]:
training_df = pl.read_csv('data/training_smiles.csv')

## Feature engineering

In [ ]:
features = ["MolWeight", "Fragments", "Lipinski-HAcount", "LogP", "TPSA", "HBD", "HBA"]

In [ ]:
from rdkit.Chem import rdMolDescriptors as d
import rdkit.Chem.Fragments as f
from rdkit.Chem import Lipinski as l
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
# from rdkit.Chem import rdFingerprintGenerator

# Define a function to calculate all features
def calculate_all_features(smiles):
    # generator = GetMorganGenerator(radius=2, fpSize=1024)

    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, None, None# , None
    
    mol_weight = d.CalcExactMolWt(mol)
    #print(f"molwt {mol_weight}")
    fragments = f.fr_Al_COO(mol)
    #print(f"fragments {fragments}")
    lipinski = l.HeavyAtomCount(mol)
    #print(f"lipinski {lipinski}")
    logp, _ = d.CalcCrippenDescriptors(mol)
    #print(f"logp {logp}")
    tpsa = d.CalcTPSA(mol)
    #print(f"tpsa {tpsa}")
    hbd = d.CalcNumHBD(mol)
    #print(f"hbd {hbd}")
    hba = d.CalcNumHBA(mol)
    #print(f"hba {hba}")
    
    # fingerprint = np.array(generator.GetFingerprint(mol))

    return mol_weight, fragments, lipinski, logp, tpsa, hbd, hba # , fingerprint

# Apply the function to the SMILES column and unpack the results
training_df = training_df.with_columns([
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[0], return_dtype=pl.Float64).alias(features[0]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[1], return_dtype=pl.Int64).alias(features[1]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[2], return_dtype=pl.Int64).alias(features[2]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[3], return_dtype=pl.Float64).alias(features[3]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[4], return_dtype=pl.Float64).alias(features[4]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[5], return_dtype=pl.Int64).alias(features[5]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[6], return_dtype=pl.Int64).alias(features[6]),
    # pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[3], return_dtype=pl.String).alias("MorganFingerprints")
])

In [ ]:
training_df.select(features)

In [ ]:
training_df.select(features).corr()

In [ ]:
# Optional: Save the training features df
training_df.write_parquet("enriched_training_df.parquet")

## Make prediction

In [ ]:
from sklearn.model_selection import train_test_split

X = training_df[features]
y = training_df[['ACTIVE']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, stratify=y)

In [ ]:
value_counts = training_df['ACTIVE'].value_counts()
print("Raw counts:")
print(value_counts)

# Actual ratio
ratio = (value_counts.filter(pl.col("ACTIVE") == 0.0)["count"] / value_counts.filter(pl.col("ACTIVE") == 1.0)["count"]).item()
print("\nCalculated ratio (negative/positive):")
print(ratio)

In [ ]:
# Use GridSearchCV to find the best parameters for XGBoost
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

params = {
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [5, 10, 20, 50, 100],
    # 'scale_pos_weight': [1, ratio],
    'objective': ['binary:logistic', 'binary:hinge', 'binary:logitraw']
}
grid_search = GridSearchCV(estimator=XGBClassifier(random_state=42, eval_metric='auc'), param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_params

In [ ]:
from xgboost import XGBClassifier

best_params = {'learning_rate': 0.1,
 'max_depth': 11,
 'n_estimators': 100,
 'objective': 'binary:logistic'}

# Taken from https://xgboost.readthedocs.io/en/stable/get_started.html
bst = XGBClassifier(
    **best_params,
    scale_pos_weight=ratio,
    random_state=42,
    eval_metric='auc'
)

In [ ]:
X_train, y_train

In [ ]:
bst.fit(X_train, y_train)

In [ ]:
# GridSearchCV of Logistic Regression
#from sklearn.linear_model import LogisticRegression
#from sklearn.model_selection import GridSearchCV

#class_weights = {0.0: 1, 1.0: ratio}

#params = {
#    'penalty': ['l1', 'l2', 'elasticnet'],
#    'C': [0.01, 0.1, 1, 10, 100],
#    'fit_intercept': [True, False],
#    'solver': ['lbfgs', 'liblinear', 'newton-cholesky', 'sag', 'saga'],
    # 'class_weight': [class_weights, 'balanced', None]
#}
#grid_search = GridSearchCV(estimator=LogisticRegression(random_state=42), param_grid=params, cv=5, scoring='roc_auc')
#grid_search.fit(X_train, y_train)
#best_params = grid_search.best_params_
#best_params

In [ ]:
from sklearn.linear_model import LogisticRegression

class_weights = {0.0: 1, 1.0: ratio}

best_params = {'C': 0.1,
 # 'class_weight': None,
 'fit_intercept': True,
 'penalty': 'l1',
'solver': 'liblinear'}

clf = LogisticRegression(**best_params, random_state=42, class_weight=class_weights)

In [ ]:
clf.fit(X_train, y_train)

## Evaluation

### Logistic Regression

In [ ]:
clf_train_preds = clf.predict(X_train)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, clf_train_preds)
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line for random classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
val_preds = rf = clf.predict(X_test)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, val_preds)
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line for random classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

### XGBoost

In [ ]:
train_preds = bst.predict(X_train)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_train, train_preds)
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line for random classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
val_preds = bst.predict(X_test)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test, val_preds)
auc = metrics.auc(fpr, tpr)
auc

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', label=f'ROC curve (AUC = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Diagonal line for random classifier
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

## Estimation + writing to file

In [ ]:
test_df = pl.read_csv('data/test_smiles.csv')
test_df

In [ ]:
test_df = test_df.with_columns([
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[0], return_dtype=pl.Float64).alias(features[0]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[1], return_dtype=pl.Int64).alias(features[1]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[2], return_dtype=pl.Int64).alias(features[2]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[3], return_dtype=pl.Float64).alias(features[3]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[4], return_dtype=pl.Float64).alias(features[4]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[5], return_dtype=pl.Int64).alias(features[5]),
    pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[6], return_dtype=pl.Int64).alias(features[6]),
    # pl.col("SMILES").map_elements(lambda x: calculate_all_features(x)[3], return_dtype=pl.String).alias("MorganFingerprints")
])

In [ ]:
train_preds = bst.predict_proba(test_df[features])

We have excluded a part of the training set to be our validation set, and we thus expect the AUC in the test set to match it. Based on our best performing model - the XGBooster model, the value should be **0.5944**.

In [ ]:
## Alternatively, we can look at the OOB performance - but it might benefit from overfitting, 
# due to it not being fully indepedent (in contrast to validation set performance).
best_params = {'learning_rate': 0.1,
 'max_depth': 11,
 'n_estimators': 50,
 'objective': 'binary:logistic'}

bst = XGBClassifier(
    **best_params,
    scale_pos_weight=ratio,
    random_state=42,
    eval_metric='auc',
    # enable_categorical=True,  # Enable categorical features
    subsample=0.8,
)

bst.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import roc_auc_score
oob_predictions = bst.predict_proba(X_train)[:, 1]
oob_auc = roc_auc_score(y_train, oob_predictions)
print(f"Training OOB AUC-ROC: {oob_auc}")

In [ ]:
from sklearn.metrics import roc_auc_score
val_oob_predictions = bst.predict_proba(X_test)[:, 1]
val_oob_auc = roc_auc_score(y_test, val_oob_predictions)
print(f"Validation OOB AUC-ROC: {val_oob_auc}")

### Writing to file

In [ ]:
with open("10.txt", "w") as f:
    f.write(f"{auc}\n")
    for pred in train_preds:
        positive_pred = pred[1]
        f.write(f"{positive_pred}\n")

In [ ]:
predictions_df = pd.read_csv("10.txt", header=None)
assert predictions_df.shape == (69646+1, 1)
assert np.all((predictions_df.values >= 0) & (predictions_df.values <= 1))